Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [179]:
from itertools import product
from random import random, randint, shuffle, seed, choice
from functools import reduce
import numpy as np
from scipy import sparse
from copy import copy

In [180]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points * 2654435761 + num_sets + density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets - 1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [181]:
PROBLEM_SIZE = 5000
NUM_SETS = 5000
DENSITY = 0.7
x = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, DENSITY)
print("Element at row=42 and column=42 :", x[42, 42])


Element at row=42 and column=42 : True


In [182]:
def fitness(state):
    cost = sum(state)
    valid = x[state, :].sum(axis = 0, dtype = bool).sum() 
    return valid, -cost

my tweak functions considers if we reached the gol or not and add or remove a set depending on the conditions

In [183]:
def tweak(state,valid,cost):
    new_state = copy(state)
    if(valid == PROBLEM_SIZE):
        index = randint(0, -cost - 1)
        for i in range(NUM_SETS):
            if (index==0 and new_state[i]==True):
                 new_state[i] = not new_state[i]
                 break
            if new_state[i] == True:
                index=index-1
    else:
        index=randint(0,NUM_SETS-1)
        while new_state[index]==True:
            index=randint(0,NUM_SETS-1)
        new_state[index]= not new_state[index]
    return new_state


In [184]:
current_state = [
    choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)
]
print(fitness(current_state))
valid,cost = fitness(current_state)
changes = 0
num_fitness = 0
MESA_THRESHOLD = 150

while changes<MESA_THRESHOLD:
    new_state = tweak(current_state,valid,cost)
    changes += 1
    num_fitness += 1
    if fitness(new_state) >= (valid,cost):
        current_state = new_state
        changes=0
        valid,cost = fitness(current_state)
        print(fitness(current_state))
print(f"number of fitness {num_fitness}")

(0, 0)
(3483, -1)
(4534, -2)
(4853, -3)
(4951, -4)
(4986, -5)
(4995, -6)
(4998, -7)
(4999, -8)
(5000, -9)
(5000, -8)
number of fitness 161
